## Down Sample the Sqlite Database to .csv Files Used in Element 3
This file will go through the steps to downsample the sqlite database created in MnDot_All_Data_Prep.  The downsampled data will be .csv files with information that is directly relevant to the visualization.  The downsampled data will be taken at 10min intervals.

### Note:
* The downsampling of update and weather table take a considerable amount of time

In [2]:
#Import needed libraries
import sqlite3
import datetime as dt
from dateutil.relativedelta import relativedelta
import csv

#Connect to database
database = "fhwa_mn.sqlite"
conn = sqlite3.connect(database)
c = conn.cursor()

#Set up timeframes
s = '2014-02-19'
start = dt.datetime.strptime(s, '%Y-%m-%d')
time_step = relativedelta(minutes=10)

In [3]:
%%time
#Add indexes to the tables used in downsampling

#Update Table Indexes
c.execute('CREATE INDEX update_vname_index ON "update" (VehicleName);')
c.execute('CREATE INDEX update_tstamp_index ON "update" (TimeStamp);')
c.execute('CREATE INDEX update_gps_index ON "update" (GPS_Quality);')

#Weather Table Indexes
c.execute('CREATE INDEX weater_stationID_index ON "weather" (StationID);')
c.execute('CREATE INDEX weater_obsType_index ON "weather" (ObsTypeName);')
c.execute('CREATE INDEX weater_tstamp_index ON "weather" (TimeStamp);')

#Mdtx Table Indexes
c.execute('CREATE INDEX mdtx_vname_index ON "mdtx" (VehicleName);')
c.execute('CREATE INDEX mdtx_tstamp_index ON "mdtx" (TimeStamp);')
c.execute('CREATE INDEX mdtx_gps_index ON "mdtx" (GPS_Quality);')

#Vaix Table Indexes
c.execute('CREATE INDEX vaix_vname_index ON "vaix" (VehicleName);')
c.execute('CREATE INDEX vaix_tstamp_index ON "vaix" (TimeStamp);')
c.execute('CREATE INDEX vaix_gps_index ON "vaix" (GPS_Quality);')

CPU times: user 19.8 s, sys: 2.7 s, total: 22.5 s
Wall time: 23.3 s


In [4]:
%%time
#The data has light vehicles, lets remove these
not_plow = ["204360","204362","206031","207301","209118","209134","209217","210161","210333","210335" \
            ,"211277","211548","212291","212312","212457","212458","213128","Jakin","Joe"]

#Find list of vehicle names from update table
c.execute('SELECT DISTINCT VehicleName FROM "update" WHERE VehicleName NOT IN ('+', '.join(['"{}"'.format(value) for value in not_plow])+')')
r = c.fetchall()
update_vehicles = [x[0] for x in r]

#Find list of weather nodes from weather table
c.execute('SELECT DISTINCT StationID FROM weather')
r = c.fetchall()
weather_nodes = [x[0] for x in r]

#Find list of vehicle names from mdtx table
c.execute('SELECT DISTINCT VehicleName FROM mdtx WHERE VehicleName NOT IN ('+', '.join(['"{}"'.format(value) for value in not_plow])+')')
r = c.fetchall()
mdtx_vehicles = [x[0] for x in r]

#Find list of vehicle names from vaix table
c.execute('SELECT DISTINCT VehicleName FROM vaix WHERE VehicleName NOT IN ('+', '.join(['"{}"'.format(value) for value in not_plow])+')')
r = c.fetchall()
vaix_vehicles = [x[0] for x in r]

CPU times: user 2.14 s, sys: 67.6 ms, total: 2.21 s
Wall time: 2.21 s


In [11]:
%%time
#Select all the weather data from the database
command = """
SELECT StationID, ObsTypeName, Timestamp, Latitude, Longitude, Observation
FROM weather
WHERE StationID = ? AND ObsTypeName = ? AND Timestamp >= ? AND Timestamp <= ?
ORDER BY Timestamp ASC
LIMIT 1
"""
ObsType = ['precipIntensity', 'essSurfaceStatus']
all_weather_lists = []
#For every weather sensor tower
for w in weather_nodes:
    #For the two observation types we are interested in
    for j in ObsType:
        my_weather_list = []
        #For every 10min interval
        for i in range(577):
            s = start + time_step*i
            e = start + time_step*(i+1)
            c.execute(command, (w, j, s, e))
            r = c.fetchone()
            if r is not None:
                my_weather_list.append(r)
        all_weather_lists.append(my_weather_list)
    
#Write the selected data to a .csv file
headers=['StationID', 'ObsTypeName', 'Timestamp', 'Latitude', 'Longitude', 'Observation']
with open('data/weather_data.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for l in all_weather_lists:
        for row in l:
            writer.writerow(row)

CPU times: user 1h 25min 9s, sys: 9min 25s, total: 1h 34min 34s
Wall time: 1h 34min 34s


In [ ]:
%%time
#Select all the data from the update table
command = """
SELECT VehicleName, Timestamp, Latitude, Longitude 
FROM 'update' 
WHERE VehicleName = ? AND Timestamp >= ? AND Timestamp <= ? AND GPS_Quality != 0
ORDER BY Timestamp ASC
LIMIT 1
"""
all_vehicle_lists = []
#For every vehicle
for v in update_vehicles:
    my_vehicle_list = []
    #For every 10min interval
    for i in range(577):
        s = start + time_step*i
        e = start + time_step*(i+1)
        c.execute(command, (v, s, e))
        r = c.fetchone()
        if r is not None:
            my_vehicle_list.append(r)
    all_vehicle_lists.append(my_vehicle_list)
    
#Write the selected data to a .csv file
headers=['VehicleName', 'Timestamp', 'Latitude', 'Longitude']
with open('vehicle_data.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for l in all_vehicle_lists:
        for row in l:
            print(row)
            writer.writerow(row)
            
print('done')

In [9]:
%%time
#Select all the data from the mdtx table
command = """
SELECT VehicleName, Timestamp, Latitude, Longitude, RoadCondition
FROM 'mdtx' 
WHERE VehicleName = ? AND Timestamp >= ? AND Timestamp <= ? AND GPS_Quality != 0
ORDER BY Timestamp ASC
LIMIT 1
"""
all_mdtx_lists = []
#For every vehicle
for v in mdtx_vehicles:
    my_mdtx_list = []
    #For every 10min interval
    for i in range(577):
        s = start + time_step*i
        e = start + time_step*(i+1)
        c.execute(command, (v, s, e))
        r = c.fetchone()
        if r is not None:
            my_mdtx_list.append(r)
    all_mdtx_lists.append(my_mdtx_list)
    
#Write the selected data to a .csv file
headers=['VehicleName', 'Timestamp', 'Latitude', 'Longitude', 'RoadCondition']
with open('data/mdtx_data.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for l in all_mdtx_lists:
        for row in l:
            writer.writerow(row)

CPU times: user 17.2 s, sys: 1.62 s, total: 18.8 s
Wall time: 19 s


In [10]:
%%time
#Select all the data from the vaix table
command = """
SELECT VehicleName, Timestamp, Latitude, Longitude, RoadTemp
FROM 'vaix' 
WHERE VehicleName = ? AND Timestamp >= ? AND Timestamp <= ? AND GPS_Quality != 0
ORDER BY Timestamp ASC
LIMIT 1
"""
all_vaix_lists = []
#For every vehicle
for v in vaix_vehicles:
    my_vaix_list = []
    #For every 10min interval
    for i in range(577):
        s = start + time_step*i
        e = start + time_step*(i+1)
        c.execute(command, (v, s, e))
        r = c.fetchone()
        if r is not None:
            my_vaix_list.append(r)
    all_vaix_lists.append(my_vaix_list)
    
#Write the selected data to a .csv file
headers=['VehicleName', 'Timestamp', 'Latitude', 'Longitude', 'RoadTemp']
with open('data/vaix_data.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for l in all_vaix_lists:
        for row in l:
            writer.writerow(row)

CPU times: user 2.83 s, sys: 350 ms, total: 3.18 s
Wall time: 3.19 s
